# Instalar dependências

In [ ]:
!pip install requests beautifulsoup4 lxml pandas python-dateutil

# Banco de dados (SQLite)

In [ ]:
import sqlite3
from pathlib import Path

DB_PATH = Path("/content/labnoticias_reportagens.db")

CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS labnoticias_reportagens (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    subtitle TEXT,
    url TEXT NOT NULL UNIQUE,
    section TEXT,
    city TEXT,
    author TEXT,
    published_at TEXT,
    updated_at TEXT,
    text TEXT,
    tags TEXT,
    raw_html TEXT,
    scraped_at TEXT NOT NULL
);
"""

def get_conn():
    DB_PATH.parent.mkdir(parents=True, exist_ok=True)
    conn = sqlite3.connect(DB_PATH)
    conn.execute("PRAGMA journal_mode=WAL;")
    conn.execute("PRAGMA foreign_keys = ON;")
    return conn

def init_db():
    conn = get_conn()
    with conn:
        conn.execute(CREATE_TABLE_SQL)
    conn.close()

def insert_labnoticias(conn, record: dict):
    sql = """
    INSERT OR IGNORE INTO labnoticias_reportagens
    (title, subtitle, url, section, city, author,
     published_at, updated_at, text, tags, raw_html, scraped_at)
    VALUES (:title, :subtitle, :url, :section, :city, :author,
            :published_at, :updated_at, :text, :tags, :raw_html, :scraped_at)
    """
    with conn:
        conn.execute(sql, record)


# HTTP + parse de datas

In [ ]:
import requests
from bs4 import BeautifulSoup
from dateutil import parser as dateparser
from datetime import datetime, timedelta, timezone
import time
import json

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; LabNoticiasScraper/1.0; +https://seu-email-ou-site)"
}

MONTHS_PT = [
    "janeiro", "fevereiro", "março", "marco", "abril", "maio",
    "junho", "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"
]

def get_soup(url: str) -> BeautifulSoup:
    resp = requests.get(url, headers=HEADERS, timeout=15)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "lxml")

def parse_datetime(text: str):
    if not text:
        return None
    try:
        return dateparser.parse(text)
    except Exception:
        return None

def extract_date_text(soup: BeautifulSoup):
    """
    Tenta achar data em <time> ou em texto com mês por extenso.
    """
    # tenta primeiro <time>
    time_tag = soup.find("time")
    if time_tag:
        if time_tag.has_attr("datetime"):
            return time_tag["datetime"]
        txt = time_tag.get_text(strip=True)
        if txt:
            return txt

    # fallback: qualquer string com mês + número
    for s in soup.stripped_strings:
        txt = s.strip()
        lower = txt.lower()
        if any(m in lower for m in MONTHS_PT) and any(ch.isdigit() for ch in lower):
            return txt
    return None


# Helpers: autor, texto, tags, cidade

In [ ]:
from bs4.element import NavigableString
import re

def extract_author_lab(soup: BeautifulSoup):
    """
    Estratégia:
    1) a[rel='author'] / span[rel='author'] / .author etc.
    2) Byline perto do título e da data.
    """
    selector_candidates = [
        'a[rel="author"]',
        'span[rel="author"]',
        '[itemprop="author"]',
        '.author',
        '.post-author',
        '.entry-author',
        '.single__author',
        '.meta-author',
    ]
    for sel in selector_candidates:
        el = soup.select_one(sel)
        if el:
            txt = el.get_text(strip=True)
            if txt:
                return txt

    date_txt = extract_date_text(soup)
    title_tag = soup.find("h1")
    if not title_tag:
        return None

    for el in title_tag.next_elements:
        if isinstance(el, NavigableString):
            txt = el.strip()
            if not txt:
                continue
            lower = txt.lower()

            if date_txt and txt.strip() == date_txt.strip():
                break
            if any(m in lower for m in MONTHS_PT) and any(ch.isdigit() for ch in lower):
                break

            if any(w in lower for w in ["compartilhar", "comentários", "comments"]):
                break

            if any(ch.isdigit() for ch in txt):
                continue

            if 2 <= len(txt) <= 60:
                return txt

        elif hasattr(el, "get_text"):
            txt = el.get_text(strip=True)
            if not txt:
                continue
            lower = txt.lower()

            if date_txt and txt.strip() == date_txt.strip():
                break
            if any(m in lower for m in MONTHS_PT) and any(ch.isdigit() for ch in lower):
                break

            if any(w in lower for w in ["compartilhar", "comentários", "comments"]):
                break

            if any(ch.isdigit() for ch in txt):
                continue

            if 2 <= len(txt) <= 60:
                return txt

    return None


def extract_text_lab(soup: BeautifulSoup) -> str | None:
    """
    Extrai o corpo da reportagem.
    Tenta:
      - <article> com .entry-content
      - ou div.entry-content
      - fallback: site inteiro (melhor que nada)
    """
    article = soup.find("article")
    container = None

    if article:
        # tenta um filho com classe comum
        for cls in ["entry-content", "post-content", "single-content", "content-area"]:
            div = article.find("div", class_=cls)
            if div:
                container = div
                break

    if not container:
        for cls in ["entry-content", "post-content", "single-content", "content-area"]:
            div = soup.find("div", class_=cls)
            if div:
                container = div
                break

    if not container:
        container = article or soup

    paragraphs = []
    for el in container.find_all(["p", "h2", "h3"], recursive=True):
        txt = el.get_text(" ", strip=True)
        if not txt:
            continue
        lower = txt.lower()

        if "leia também" in lower:
            break
        if any(w in lower for w in ["compartilhar", "comentários", "comments"]):
            continue

        paragraphs.append(txt)

    if not paragraphs:
        return None

    text = "\n\n".join(dict.fromkeys(paragraphs))
    return text


def extract_tags_lab(soup: BeautifulSoup):
    """
    Tags a partir de:
      - rel='tag'
      - ou links /tag/
    """
    tags = []
    for a in soup.select("a[rel='tag'], a[href*='/tag/']"):
        txt = a.get_text(strip=True)
        if txt:
            tags.append(txt)
    return list(dict.fromkeys(tags))


def extract_city_from_text(paragraphs):
    """
    Igual G1/Jornal Opção: tenta 'Cidade - ' na primeira linha.
    """
    if not paragraphs:
        return None
    first_line = paragraphs[0]
    if " - " in first_line:
        maybe = first_line.split(" - ")[0]
        if len(maybe.split()) <= 4:
            return maybe
    return None


# Listagem de reportagens /page/N/

A categoria de reportagens provavelmente funciona assim:

https://labnoticias.jor.br/category/reportagens/

https://labnoticias.jor.br/category/reportagens/page/2/

https://labnoticias.jor.br/category/reportagens/page/3/

etc.

In [ ]:
BASE_LAB_URL = "https://labnoticias.jor.br/category/reportagens/"

def fetch_lab_listing(page: int):
    """
    Devolve lista de URLs de reportagens em uma página do arquivo.

    page=1 -> /category/reportagens/
    page>1 -> /category/reportagens/page/{page}/
    """
    if page == 1:
        url = BASE_LAB_URL
    else:
        url = f"{BASE_LAB_URL}page/{page}/"

    print(f"[LIST] Página {page}: {url}")
    soup = get_soup(url)

    urls = set()

    # 1) Tenta pegar os cards de post via <article> + título
    for art in soup.find_all("article"):
        h = art.find(["h2", "h3"])
        if not h:
            continue
        a = h.find("a")
        if not a or not a.get("href"):
            continue
        href = a["href"].strip()

        # só URLs do próprio site
        if not href.startswith("https://labnoticias.jor.br/"):
            continue

        # DESCARTA coisas que não são post de reportagem:
        # - páginas de autor, categoria, tag, paginação etc.
        if any(p in href for p in ["/author/", "/category/", "/tag/", "/page/"]):
            continue

        urls.add(href)

    # 2) Fallback: se por algum motivo não veio nada, fazer uma varredura genérica,
    #    mas SEM /author/, /category/, /tag/, /page/
    if not urls:
        for a in soup.select("a"):
            href = (a.get("href") or "").strip()
            if not href.startswith("https://labnoticias.jor.br/"):
                continue
            if any(p in href for p in ["/author/", "/category/", "/tag/", "/page/"]):
                continue
            urls.add(href)

    urls = sorted(urls)
    print(f"[LIST] Encontradas {len(urls)} URLs de matérias na página {page}.")
    return urls


# Extrair uma reportagem (record completo)

In [ ]:
def fetch_lab_article(url: str) -> dict:
    """
    Baixa e extrai dados de uma reportagem do Lab Notícias
    no mesmo esquema de colunas do G1.
    """
    print(f"[ARTICLE] Baixando: {url}")
    soup = get_soup(url)

    # título
    title_tag = soup.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else None

    # subtítulo (se tiver, pode ser em h2 ou parágrafo destacado;
    # por enquanto deixo None; se achar padrão depois, dá pra ajustar)
    subtitle = None

    # autor
    author = extract_author_lab(soup)

    # data
    published_raw = extract_date_text(soup)
    published_iso = None
    dt = parse_datetime(published_raw) if published_raw else None
    if dt is not None:
        published_iso = dt.isoformat()

    updated_at = None  # se o site marcar atualização, dá pra estender depois

    # seção (é sempre Reportagens aqui)
    section = "Reportagens"

    # texto
    text = extract_text_lab(soup)

    # city (heurística)
    paragraphs = text.split("\n\n") if text else []
    city = extract_city_from_text(paragraphs)

    # tags
    tags_list = extract_tags_lab(soup)
    tags_str = json.dumps(tags_list, ensure_ascii=False) if tags_list else None

    raw_html = str(soup)
    scraped_at = datetime.now(timezone.utc).isoformat()

    return {
        "title": title,
        "subtitle": subtitle,
        "url": url,
        "section": section,
        "city": city,
        "author": author,
        "published_at": published_iso or published_raw,
        "updated_at": updated_at,
        "text": text,
        "tags": tags_str,
        "raw_html": raw_html,
        "scraped_at": scraped_at,
    }


# Crawler principal (com corte de meses, se quiser)

In [ ]:
def crawl_labnoticias_reportagens(max_pages: int = 30, months: int = 6):
    """
    Percorre as páginas da categoria 'Reportagens' do Lab Notícias
    e salva apenas matérias dos últimos `months` meses (aprox).
    """
    init_db()
    conn = get_conn()

    cutoff = datetime.now(timezone.utc) - timedelta(days=30 * months)
    print(f"[INFO] Data de corte aproximada: {cutoff.date()} (últimos {months} meses)")

    seen = set()
    pages_only_old = 0

    for page in range(1, max_pages + 1):
        try:
            article_urls = fetch_lab_listing(page)
        except Exception as e:
            print(f"[STOP] Erro ao listar página {page}: {e}")
            break

        if not article_urls:
            print(f"[STOP] Página {page} sem matérias, encerrando.")
            break

        page_has_newer = False

        for url in article_urls:
            if url in seen:
                continue
            seen.add(url)

            try:
                record = fetch_lab_article(url)
            except Exception as e:
                print(f"[ERROR] {url}: {e}")
                continue

            if not record.get("title") or not record.get("text"):
                print(f"[WARN] Sem título ou texto, pulando: {url}")
                continue

            pub_str = record.get("published_at")
            pub_dt = parse_datetime(pub_str) if pub_str else None

            if pub_dt is not None:
                if pub_dt.date() < cutoff.date():
                    print(
                        f"[SKIP old] {record['title'][:60]}... "
                        f"({pub_dt.date()}) < corte ({cutoff.date()})"
                    )
                    continue
                else:
                    page_has_newer = True
            else:
                page_has_newer = True  # sem data parseável, guardo mesmo assim

            insert_labnoticias(conn, record)
            print(f"[OK] {record['title'][:80]}")
            time.sleep(1)

        if not page_has_newer:
            pages_only_old += 1
            print(f"[INFO] Página {page} só com notícia velha.")
            if pages_only_old >= 3:
                print(
                    f"[STOP] {pages_only_old} páginas seguidas só com notícia < corte. Encerrando."
                )
                break
        else:
            pages_only_old = 0

        time.sleep(1)

    conn.close()
    print("[DONE] Crawl Lab Notícias finalizado.")


# Para rodar

In [ ]:
crawl_labnoticias_reportagens(max_pages=20, months=6)


[INFO] Data de corte aproximada: 2025-06-13 (últimos 6 meses)
[LIST] Página 1: https://labnoticias.jor.br/category/reportagens/
[LIST] Encontradas 15 URLs de matérias na página 1.
[ARTICLE] Baixando: https://labnoticias.jor.br/
[OK] Lab Notícias
[ARTICLE] Baixando: https://labnoticias.jor.br/2025/11/
[OK] Mês:novembro 2025
[ARTICLE] Baixando: https://labnoticias.jor.br/2025/11/28/a-floresta-que-vale-bilhoes-como-o-mercado-de-creditos-de-carbono-virou-desejo-empresarial/
[OK] A floresta que vale bilhões: como o mercado de créditos de carbono virou desejo 
[ARTICLE] Baixando: https://labnoticias.jor.br/2025/11/28/aparecida-de-goiania-62-anos-de-emancipacao-da-segunda-maior-cidade-de-goias/
[OK] Aparecida de Goiânia: 62 anos de emancipação da segunda maior cidade de Goiás
[ARTICLE] Baixando: https://labnoticias.jor.br/2025/11/28/entre-o-25-de-marco-e-o-20-de-novembro-o-ceara-celebra-sua-abolicao-pioneira-mas-a-populacao-sabe-disso/
[OK] Entre o 25 de março e o 20 de novembro: o Ceará cele

In [ ]:
import pandas as pd

conn = get_conn()
df = pd.read_sql_query("""
    SELECT *
    FROM labnoticias_reportagens
    ORDER BY id DESC
""", conn)
conn.close()

df


,id,title,subtitle,url,section,city,author,published_at,updated_at,text,tags,raw_html,scraped_at
0,200,‘Lei não trata pessoas gestantes como capazes ...,None,https://labnoticias.jor.br/2024/01/24/perpetua...,Reportagens,None,ANDRESSA MOREIRA BUENO,- 24 de janeiro de 2024,None,Aprovada no dia 11 de janeiro de 2024 pela Ass...,"[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:58:54.873577+00:00
1,199,Ódio contra crianças: por que um discurso aceito?,None,https://labnoticias.jor.br/2024/01/24/odio-con...,Reportagens,None,Natália EduardaAcadêmica de Jornalismo | UFG,- 24 de janeiro de 2024,None,O comentário capturado em um perfil no Instagr...,"[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:58:53.149560+00:00
2,198,NOSSO LAR 2: Filme baseado na obra de Chico Xa...,None,https://labnoticias.jor.br/2024/01/24/nosso-lar/,Reportagens,None,FRANCISCA PATRICIA DE SOUZA SILVAestudante de ...,- 11 de agosto de 2024,None,Dia 25 de janeiro chega nas telonas “Nosso Lar...,"[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:58:51.469383+00:00
3,197,Moda sustentável: como Goiânia está incorporan...,None,https://labnoticias.jor.br/2024/01/24/moda-sus...,Reportagens,None,Ana Beatriz MilhomemEstudante de jornalismo pe...,- 24 de janeiro de 2024,None,Cotidianamente lojas nos bombardeiam com novas...,"[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:58:49.827851+00:00
4,196,Janeiro Branco: a influência do vicio tecnológ...,None,https://labnoticias.jor.br/2024/01/24/janeiro-...,Reportagens,None,Julhia Mendes BritoEstudante de Jornalismo na ...,- 24 de janeiro de 2024,None,JANEIRO BRANCO\n\nO Janeiro Branco é uma campa...,"[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:58:48.084783+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,5,Entre o 25 de março e o 20 de novembro: o Cear...,None,https://labnoticias.jor.br/2025/11/28/entre-o-...,Reportagens,None,Tiago CalebEstudante de Jornalismo na UFG. Apa...,Entre o 25 de março e o 20 de novembro: o Cear...,None,"No dia 25 de março, o estado do Ceará celebra ...","[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:52:17.037024+00:00
196,4,Aparecida de Goiânia: 62 anos de emancipação d...,None,https://labnoticias.jor.br/2025/11/28/aparecid...,Reportagens,None,Fernando SilveiraEstudante de Jornalismo na Un...,Aparecida de Goiânia: 62 anos de emancipação d...,None,"Foi no ano de 1922, com a construção de uma ig...","[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:52:15.303088+00:00
197,3,A floresta que vale bilhões: como o mercado de...,None,https://labnoticias.jor.br/2025/11/28/a-flores...,Reportagens,None,Luiza GuimarãesEstudante de jornalismo na UFG....,- 28 de novembro de 2025,None,Como o mercado de créditos de carbono virou di...,"[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:52:13.704379+00:00
198,2,Mês:novembro 2025,None,https://labnoticias.jor.br/2025/11/,Reportagens,None,Mês:,novembro 2025 - Lab Notícias,None,"Nos últimos anos, Goiânia vem passando por uma...","[""educação"", ""esportes"", ""economia"", ""cultura""...","<!DOCTYPE html>\n<html lang=""pt-BR"">\n<head>\n...",2025-12-10T19:52:12.029530+00:00


In [ ]:
df.to_csv("lab_noticias.csv")